In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1, data processing: <br>
    with a dataframe:

In [ ]:
data = pd.read_csv('../input/sample/wn21630project_lyrics_input.csv')
data = data[['singer','title','lyrics']]
section_list = ['intro','verse','chorus','bridge','hook','outro','pre_chorus','post_chorus']
lyric_aftercleaning={'title':[]}
for i in section_list:
    lyric_aftercleaning[i] = []


In [ ]:
data['lyrics'] = data['lyrics'].apply(lambda x: x.lower())
data['lyrics'] = data['lyrics'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
processed_data = pd.DataFrame(lyric_aftercleaning)
processed_data = pd.merge(processed_data,data,on='title',how='right')
processed_data = processed_data.drop_duplicates('lyrics','first')
processed_data.fillna('',inplace = True)
processed_data.reset_index(drop=True, inplace=True)


Separate the lyrics by its structure

In [ ]:
section = ''
tempstr = ''
for index, row_i in processed_data.iterrows():
    text_start_flag = False
    section_start_flag = False
    doc = re.split('(\[|\])',row_i.lyrics)
    for token in doc:
        if token =="[":
            pre_or_post = ''
            text_start_flag = False
            section_start_flag = True
            continue
        if token == ']':
            text_start_flag = True
            section_start_flag = False
            continue
        if section_start_flag:
            a = re.split(' ',token)
            if a[0] in section_list:
                section = a[0]
                continue
            if 'post' in token:
                a1 = re.split(' ',re.sub('post-*','',token))
                if a1[0]=='chorus': section = 'post_chorus'
                elif a1[0] in section: section =a1[0]
            if 'pre' in a:
                a1 = re.split(' ',re.sub('pre-*','',token))
                if a1[1]=='chorus': section = 'pre_chorus'
                elif a1[1] in section: section =a1[1]        
            else: section = '' 
            continue
        if text_start_flag and section !='':
            processed_data.iloc[index][section] = processed_data.iloc[index][section] + re.sub('\n', ' <Newline> ', token)
        
            

In [ ]:
processed_data.iloc[0].lyrics

In [ ]:
processed_data.to_csv('wn21630project_input_processed.csv')

# **Generating Topical Poetry**

In [ ]:
#https://github.com/tensorflow/nmt


# **GPT-2: no structure**

In [ ]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2' )

In [ ]:
d = list(processed_data.lyrics.apply(lambda x : re.sub('\n',' <Newline> ',x)))
length = len(d)
with open('./train_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(d[:int(length*0.6)]))
with open('./dev_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(d[int(length*0.6):int(length*0.8)]))
with open('./test_dataset.txt','w') as f:
    f.write(' <Newsong> '.join(d[int(length*0.8):]))

In [ ]:
train_path = './train_dataset.txt'
dev_path = './dev_dataset.txt'

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(dev_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=32)
        #   block_size=128)
     
    dev_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=dev_path,
          block_size=32) 
        #   block_size=128)  
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,dev_dataset,data_collator

train_dataset,dev_dataset,data_collator = load_dataset(train_path,dev_path,tokenizer)



In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')


training_args = TrainingArguments(
    output_dir="./output", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=4, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    # per_device_train_batch_size=16, # batch size for training
    # per_device_eval_batch_size=32,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

In [ ]:
trainer.train()
trainer.save_model()


In [ ]:
from transformers import pipeline
from transformers import GPT2Tokenizer,GPT2LMHeadModel
import torch
device ='cuda'
model_new = GPT2LMHeadModel.from_pretrained('./output').to(device)
model.eval()

def generation(prompt,num):
    model_new.eval()



    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    print(generated)

    sample_outputs = model_new.generate(generated, do_sample=True,   top_k=50, max_length = 800,top_p=0.95, num_return_sequences=1
                                    )
    dict_new = {}
    for i, sample_output in enumerate(sample_outputs):
        
        a = tokenizer.decode(sample_output, skip_special_tokens=True)
        dict_new[num] = re.sub('<Newline>|[,*]','\n',a)
        num +=1
        #print("{}: {}\n\n".format(i, re.sub('\sL+','\n',a)))
    return dict_new




In [ ]:
x = {'1':'1'}
list(x.values())[0]

In [ ]:
generation('I love',1).values()

In [ ]:
text_gen = pipeline('text-generation',model=model, tokenizer=GPT2Tokenizer.from_pretrained('gpt2'),config={'max_length':8000})

In [ ]:
m = re.match("\[intro.*\] <Newline>.{20}?",d[0])

In [ ]:
import json
new_songs = []
num = 0
for i in d[int(length*0.8):]:
    try:
        m = re.match("\[intro.*\] <Newline>.{20}?",i)
        prompt = re.sub('\[.*\] <Newline>','',m.group())
        if m == '': prompt = 'I only love'
        new = generation(prompt,num)
        new_songs.append(list(new.values()))
    except:
        prompt = 'be a savage'
        new = generation(prompt,num)
        new_songs.append(list(new.values()))
    num +=1
    print(num)
new_dict = {'GPT2-no-structure':new_songs}


In [ ]:
with open("./output_song_baseline.json","w") as f:
    json.dump(new_dict,f)
    print("加载入文件完成...")